In [1]:
import torch
from torch import nn
import numpy as np
import torch.nn.functional as F

from torch.distributions.uniform import Uniform

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd

from torch.utils.tensorboard import SummaryWriter

import time

from IPython.display import display, clear_output

import math

from diffusion import Diffusion

/home/mvmacfarlane/miniconda3/envs/CVAE_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def graph(data_1,data_2):


    data_1 = pd.DataFrame(data_1.detach().numpy(),columns = ['x','y'])
    data_2 = pd.DataFrame(data_2.detach().numpy(),columns = ['x','y'])


    #Log latents sampling to tensorboard
    fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
    sns.scatterplot(ax=ax1,data=data_1, x="x", y="y",s=20)

    ax1.set_xlim(-15,15)
    ax1.set_ylim(-15,15)


    sns.scatterplot(ax=ax2,data=data_2, x="x", y="y",s=20)

    ax2.set_xlim(-15,15)
    ax2.set_ylim(-15,15)

    ax2.axhline(y=5, color='r', linestyle='-')
    ax2.axvline(x=5, color='r', linestyle='-')

    plt.show(fig)

    clear_output(wait = True)
    plt.pause(0.5)
    


In [3]:
#target_dist = torch.normal(mean = 1, std = 1,size = (20,2))


#add tensorboard

def train(model,name):


    writer = SummaryWriter("runs/" + name)

    batch_size = 512
    epoch_num = 20
    epoch_size = 93440

    #time_distribution = Uniform(1, 300)

    params = list(model.parameters())
    optimizer = torch.optim.Adam(params, lr=1e-3)



    for epoch_idx in range(epoch_num):

        #Train

        total_loss = 0

        for j in range(100):

            optimizer.zero_grad()

            target_dist = torch.normal(mean = 5, std = 1,size = (batch_size,2))

            loss = model.calculate_loss(x0 = target_dist)

            

            total_loss += loss.item()

            loss.backward()

            optimizer.step()

        #Evaluate
        gaussian = torch.normal(mean = 0, std = 1,size = (batch_size,2))
        diffussed_distribution = model.generate_sample(batch_size,device = "cpu")

        graph(gaussian,diffussed_distribution)


        print("Diffusion Loss :{}".format(total_loss))


        writer.add_scalar("Diffusion Loss", total_loss, epoch_idx)
        writer.add_scalar("Diffused Mean", torch.mean(diffussed_distribution), epoch_idx)
        writer.add_scalar("Diffused STD", torch.std(diffussed_distribution), epoch_idx)

        writer.flush()



In [4]:
test = Diffusion(steps = 600)

train(test,"diffusion baseline steps 600 New")

Diffusion Loss :66.89485150575638
